## Trying to Merge Django Api

Clone the repository for google Colab

In [ ]:
!git clone https://github.com/Amine-Guenaoui/OpenVoice

Go inside the necessary Folder ( if you have restarted the Session you have to go inside the folder again)

In [12]:
cd OpenVoice

e:\Github Repos\OpenVoice\OpenVoice


C:\Users\Gaming\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Download The Checkpoints Folder

In [ ]:
import requests
url = "https://myshell-public-repo-hosting.s3.amazonaws.com/checkpoints_1226.zip"
r = requests.get(url, allow_redirects=True)

open('checkpoints_1226.zip', 'wb').write(r.content)

420616031

Extract the checkpoints Folder

In [ ]:
import zipfile
with zipfile.ZipFile("checkpoints_1226.zip","r") as zip_ref:
    zip_ref.extractall("")

Install the Requirements May Require Restarting the Machine ( don't forget to go inside the folder again )

In [1]:
!python -V

Python 3.11.8


In [2]:
!pip install -r requirements.txt
!pip install --upgrade huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached faster_whisper-1.0.2-py3-none-any.whl.metadata (15 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached wavmark-0.0.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached eng_to_ipa-0.0.2.tar.gz (2.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached inflect-7.2.1-py3-none-any.whl.metadata (21 kB)
  Using cached whisper_timestamped-1.15.4-py3-none-any.whl.metadata (1.2 kB)
  Using cached openai-1.30

In [8]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

C:\Users\Gaming\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In [3]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker.
It is an average of multiple sentences generated by the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

In [4]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

The `reference_speaker.mp3` below points to the short audio clip of the reference whose voice we want to clone. We provide an example here. If you use your own reference speakers, please **make sure each speaker has a unique filename.** The `se_extractor` will save the `targeted_se` using the filename of the audio and **will not automatically overwrite.**

In [24]:
reference_speaker = 'resources/30 secs.m4a' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

OpenVoice version: v1
[(0.078, 28.754), (28.782, 30.232375)]
after vad: dur = 30.126009070294785


### Inference

In [25]:
save_path = f'{output_dir}/audioBookresult.wav'

# Run the base speaker tts
text = "Hello Farouk This is a message from Amine Local Computer USing RTX Nvida Card i hope you like the results"
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
Hello Farouk This is a message from Amine Local Computer USing RTX Nvida Card i hope you like the results
 > ===========================
hɛˈloʊ fəˈɹuk ðɪs ɪz ə ˈmɛsɪdʒ fɹəm amine* ˈloʊkəɫ kəmˈpjutəɹ ˈjuzɪŋ ɹtx* nvida* kɑɹd aɪ hoʊp ju laɪk ðə ɹɪˈzəɫts.
 length:114
 length:114


**Try with different styles and speed.** The style can be controlled by the `speaker` parameter in the `base_speaker_tts.tts` method. Available choices: friendly, cheerful, excited, sad, angry, terrified, shouting, whispering. Note that the tone color embedding need to be updated. The speed can be controlled by the `speed` parameter. Let's try whispering with speed 0.9.

In [28]:
source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)
save_path = f'{output_dir}/output_whispering.wav'

# Run the base speaker tts
text = "Hello This is Amine. I am now Speaking To Test the quality of the AUDIO, i hope the voice is more human than anything else."
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='whispering', language='English', speed=0.9)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
Hello This is Amine. I am now Speaking To Test the quality of the AUDIO,
i hope the voice is more human than anything else.
 > ===========================
hɛˈloʊ ðɪs ɪz amine*. aɪ æm naʊ ˈspikɪŋ tɪ tɛst ðə kˈwɑləti əv ðə ˈɑdiˌoʊ,
 length:74
 length:74
aɪ hoʊp ðə vɔɪs ɪz mɔɹ ˈjumən ðən ˈɛniˌθɪŋ ɛɫs.
 length:47
 length:47


**Try with different languages.** OpenVoice can achieve multi-lingual voice cloning by simply replace the base speaker. We provide an example with a Chinese base speaker here and we encourage the readers to try `demo_part2.ipynb` for a detailed demo.

In [ ]:

ckpt_base = 'checkpoints/base_speakers/ZH'
base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

source_se = torch.load(f'{ckpt_base}/zh_default_se.pth').to(device)
save_path = f'{output_dir}/output_chinese.wav'

# Run the base speaker tts
text = "今天天气真好，我们一起出去吃饭吧。"
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='Chinese', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

**Tech for good.** For people who will deploy OpenVoice for public usage: We offer you the option to add watermark to avoid potential misuse. Please see the ToneColorConverter class. **MyShell reserves the ability to detect whether an audio is generated by OpenVoice**, no matter whether the watermark is added or not.

In [7]:
!pip install django

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.2 MB 1.3 MB/s eta 0:00:07
   - -------------------------------------- 0.3/8.2 MB 2.5 MB/s eta 0:00:04
   --- ------------------------------------ 0.6/8.2 MB 3.8 MB/s eta 0:00:02
   ----- ---------------------------------- 1.1/8.2 MB 5.2 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.2 MB 6.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.1/8.2 MB 7.1 MB/s eta 0:00:01
   ------------ --------------------------- 2.6/8.2 MB 7.6 MB/s eta 0:00:01
   -------------- ------------------------- 3.0/8.2 MB 7.8 MB/s eta 0:00:01
   ---------------- ----------------------- 3.3/8.2 MB 8.1 MB/s eta 0:00:01
   ---------------- ----------------------- 3.3/8.2 MB 8.1 MB/s eta 0:00:01
   -------------------- 

In [8]:
!django-admin startproject Test_API

In [9]:
cd Test_API

e:\Github Repos\OpenVoice\Test_API


C:\Users\Gaming\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [25]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://hc9rqnusqxi-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [10]:
#!python manage.py makemigrations
!python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

 Volume in drive E is OS
 Volume Serial Number is 82DE-23F1

 Directory of e:\Github Repos\OpenVoice

06/02/2024  10:42 PM    <DIR>          .
05/30/2024  03:14 PM    <DIR>          ..
05/30/2024  03:14 PM               147 .gitignore
05/30/2024  03:32 PM    <DIR>          checkpoints
05/30/2024  03:32 PM       420,616,031 checkpoints_1226.zip
05/30/2024  03:14 PM             7,030 demo_part2.ipynb
05/30/2024  03:14 PM             4,869 demo_part3.ipynb
06/02/2024  06:37 PM            50,826 Django Merge.ipynb
06/02/2024  08:59 PM            58,709 Django_Merge.ipynb
05/30/2024  03:14 PM    <DIR>          docs
05/30/2024  03:14 PM           961,326 example_reference.mp3
06/02/2024  06:34 PM            50,841 Final Note Book For Tests.ipynb
05/30/2024  03:14 PM             1,063 LICENSE
05/30/2024  03:33 PM    <DIR>          openvoice
06/02/2024  10:40 PM    <DIR>          outputs
06/02/2024  10:40 PM    <DIR>          processed
05/30/2024  03:14 PM             3,217 README.md
05/30/202

In [11]:
!python manage.py runserver 8000

^C
